## 검색 건수 만큼 게시물의 상세내역을 추출해서 다양한 파일로 저장하기

In [ ]:
# 환경설정
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import os
import math
import random

In [ ]:
print("="*70)
print("예제: 대한민국 구석구석 사이트의 여행지 정보 수집하기")
print("="*70)

query_txt = input("크롤링할 키워드는 무엇입니까?")
cnt = int(input("크롤링할 검색 건수"))
page_cnt = math.ceil(cnt/10)

In [ ]:
f_dir = input("결과 파일들을 저장할 폴더명만 쓰세요:(C:\\Study\\Python\\notebook\\data\\)")
n = time.localtime()
s = '{}-{}-{}-{}-{}-{}' .format(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)

ff_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xlsx'

In [ ]:
s_time = time.time()
path="C:\\Study\\Python\\datadown\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://korean.visitkorea.or.kr/")
time.sleep(random.randrange(2,5))

try:
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except:
    print('코로나 창이 없습니다.')
    
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
element.send_keys("\n")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

print("="*50)
print("크롤링할 키워드{}" .format(query_txt))
print("크롤링할 총 페이지 갯수:", page_cnt)


In [ ]:
no2=[]
contents2=[]

no=1
for x in range(1,page_cnt+1):
    print("{} 페이지 내용의 수집을 시작합니다".format(x))
    
    #한페이지에 10개의 게시물이 있기때문에 range(1,11)
    for i in range(1,11):
        #요청한 검색건수에 도달하면 for loop나오기
        if no > cnt:
            break
            
        #각 게시글 제목 누르기    
        try:
            driver.find_element_by_xpath('''//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a'''%i).click()
        except:
            #ul의 li들중에 광고banner같은 것들은 skip하기위해
            continue

        time.sleep(random.randrange(2,5))
        
        html = driver.page_source # 상세페이지 HTML가져오기
        soup = BeautifulSoup(html, 'html.parser')
        #상세페이지에서 내용부분(div) 가져오기
        content_list = soup.find('div','wrap_contView')
        
        try:
        #내용태그에서 텍스트만 가져온다
            content_1 = content_list.find('div','txt_p').get_text()
        except:
            #div class이름이 txt_p가 아닌경우
            try:
                content_1 = content_list.find('div','inr').get_text()
                #content_1 = content_0.find('p').get_text()
            except:
                driver.back()
                #continue를 넣지않으면 driveback()후 다음 상세페이지로 이동하지않고 계속 현페이지에 머문다
                continue
                
        print(no, ': ', content_1) #번호와, 추출한 데이터 출력한다.
        print("\n")
        
        f = open(ff_name,'a',encoding='utf-8')
        f.write(str(no)+':'+str(content_1)+"\n")
        f.close()
        
        no2.append(no)
        contents2.append(content_1)
        
        driver.back()
        time.sleep(2)
        
        no += 1
    
    if x > page_cnt+1:
        break
    x += 1
    
    if(x%5==1):
        driver.find_element_by_link_text("다음").click()
    else:
        driver.find_element_by_link_text("{}".format(x)).click()
    time.sleep(2)

In [ ]:
jeju = pd.DataFrame()
jeju['번호']=no2
jeju['내용']=contents2

jeju.to_csv(fc_name, encoding="utf-8-sig",index=False)
jeju.to_excel(fx_name, index=False)

e_time = time.time()
d_time = e_time - s_time

In [ ]:
#요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(d_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
 
driver.close( )

In [ ]:
#########################################################
#practice 
#한번에 실행 coding from scratch:

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import math
import random
import os

###빠트린곳: report형식으로 어떤 주제의 데이터 수집인지 명시필요
print("="*80)
print("예제:대한민국 구석구석 사이트의 여행지 정보 수집하기")
print("="*80)

#데이터 검색 조건
query_txt = input('검색할 키워드 입력:')
cnt = int(input('요청 검색건수 입력:'))
cnt_per_page = 10
page_cnt = math.ceil(cnt/cnt_per_page)

#검색 결과를 저장할 디렉토리 생성
dir_name=input('저장할 폴더 경로 입력:(C:\\Study\\Python\\notebook\\data\\)')
n=time.localtime()
s="{}-{}-{}-{}-{}".format(n.tm_year,n.tm_mon,n.tm_mday,n.tm_hour,n.tm_min)
print('\n')

###빠트린곳: 디렉토리생성 코드!!!! (os를 import한 이유...)
os.makedirs(dir_name+s+'-'+query_txt)

#검색 결과를 저장 할 파일 이름 생성
ft_name=dir_name+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=dir_name+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=dir_name+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xlsx'

###빠트린곳: 검색+수집에 걸린 시간 측정위해 시작시간 확인
s_time = time.time()

#드라이버 생성 및 웹사이트 열기
path = "C:\\Study\\Python\\datadown\\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://korean.visitkorea.or.kr/")

###빠트린곳: 중간중간 time.sleep넣기
time.sleep(random.randrange(2,5))

#해당 웹사이트의 홈페이지 alert msg창이 있다면, 닫기
try:
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except:
    print('코로나 창이 없습니다')

###틀린곳: element라는 변수에 담아야함!!
#검색 키워드 입력 후 결과 페이지 조회
element = driver.find_element_by_id("inp_search")
element.send_keys(query_txt)
element.send_keys("\n")

html=driver.page_source
soup = BeautifulSoup(html,'html.parser')

###틀린곳: x(page_cnt의 카운터)별로 forloop돌기전에!! loop밖에서 빈리스트 생성해야지
no2 = []
contents2 =[]
#no 초기화. 수집 할 게시물 갯수 카운터
no=1

#검색결과 리스트페이지에서 하나씩 상세페이지를 조회하여 내용 받아오기
for x in range(1,page_cnt+1):
    #검색목록 페이지의 ul내 li중에 게시글 외에 다른게 있다면, i의 count를 올라가지만, 내용을 출력되지않는다.
    print("{}번 검색결과 페이지의 상세 내용 입니다.==============".format(x))
    
    for i in range(1,cnt_per_page+2):
        
        ###틀린곳: for-loop을 나올 코드
        #range의 숫자만큼 반복하는데 왜 break할 조건을 또 명시해야하지?
        if no > cnt:
            break
            
        ###틀린곳: driver가 element를 찾는다!!
        try:
            driver.find_element_by_xpath('//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %i).click()
        except:
            continue
        
        time.sleep(random.randrange(2,5))
        
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        
        content_list = soup.find('div','wrap_contView')
        
        ###틀린곳: .get_text()를 써야지만 내부 문자내용만 받아올수있다
        try:
            content_1 = content_list.find('div','txt_p').get_text()
        except:
            try:
                content_1 = content_list.find('div','inr').get_text()
            except:
                #아래 txt파일에 쓰는 코드를 거치지않고 바로 driver.back() 검색목록으로 뒤로가기
                driver.back()
                continue
        
        print(no,":", content_1)
        print('\n')
        
        ###틀린곳: 여기서 바로 content_1을 txt파일에 쓴다.
        f = open(ft_name,'a',encoding='utf-8')
        f.write(str(no)+':'+str(content_1)+'\n')
        f.close()
        
        no2.append(no)   
        no += 1
        contents2.append(content_1)
        
        ###틀린곳: txt파일에 상세페이지내용1개 쓰고, 리스트contents2에도 더한 후,
        #다시 검색목록 페이지로 뒤로가기
        driver.back()
        time.sleep(2)
        print('i:',i)
    
    x+=1
    
    if(x%5 ==1):
        driver.find_element_by_link_text("다음").click()
    else:
        driver.find_element_by_link_text("{}".format(x)).click()   
    time.sleep(2)
    
    
tripinfo = pd.DataFrame()
tripinfo['번호']=no2
tripinfo['내용']=contents2

tripinfo.to_csv(fc_name, encoding='utf-8-sig', index=False)
tripinfo.to_excel(fx_name, index=False)

e_time = time.time()
d_time = e_time-s_time

print("검색결과 추출 시간:{}".format(d_time))
print("저장된 검색결과 파일:\n {} \n, {} \n, {}".format(ft_name, fc_name, fx_name))

driver.close()

In [ ]:
#practice finding element tag:

content_list = soup.find('ul','list_thumType type1')
print(type(content_list))

photo = content_list.find('div','photo')
print(type(photo))
print(photo.select('a[href]'))

txt = content_list.find('div','area_txt')
print(type(txt))

popMenu = content_list.find('div','pop_subMenu')
print(type(popMenu))

In [ ]:
driver.close()